In [2]:
import pandas as pd

In [3]:
raw_df=pd.read_csv('insurance.csv')

In [4]:
raw_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
inputs=raw_df[['age','sex','bmi','children','smoker','region']]
target=raw_df['charges']

In [6]:
numerical_cols=['age','bmi','children']

In [7]:
from sklearn.model_selection import train_test_split
inputs_train,input_test,target_train,target_test=train_test_split(inputs, target, test_size=0.25, random_state=42)

In [8]:
dict1={'no':0,'yes':1}

In [9]:
dict2={'male':1,'female':0}

In [10]:
dict3={'southeast':1,'southwest':2,'northeast':3,'northwest':4}

In [11]:
inputs_train['smoker']=inputs_train.smoker.map(dict1)
input_test['smoker']=input_test.smoker.map(dict1)

In [12]:
inputs_train['sex']=inputs_train.sex.map(dict2)
input_test['sex']=input_test.sex.map(dict2)

In [13]:
inputs_train['region']=inputs_train.region.map(dict3)
input_test['region']=input_test.region.map(dict3)

In [14]:
input_test

,age,sex,bmi,children,smoker,region
764,45,0,25.175,2,0,3
887,36,0,30.020,0,0,4
890,64,0,26.885,0,1,4
1293,46,1,25.745,3,0,4
259,19,1,31.920,0,1,4
...,...,...,...,...,...,...
342,60,0,27.550,0,0,3
308,58,1,34.865,0,0,3
1128,34,1,32.800,1,0,2
503,19,1,30.250,0,1,1


In [15]:
inputs_train

,age,sex,bmi,children,smoker,region
693,24,1,23.655,0,0,4
1297,28,0,26.510,2,0,1
634,51,1,39.700,1,0,2
1022,47,1,36.080,1,1,1
178,46,0,28.900,2,0,2
...,...,...,...,...,...,...
1095,18,0,31.350,4,0,3
1130,39,0,23.870,5,0,1
1294,58,1,25.175,0,0,3
860,37,0,47.600,2,1,2


In [16]:
from sklearn.ensemble import RandomForestRegressor

In [17]:
model=RandomForestRegressor(random_state=42)

In [18]:
model.fit(inputs_train,target_train)

RandomForestRegressor(random_state=42)

In [19]:
preds=model.predict(inputs_train)

In [20]:
preds

array([ 3507.1931982,  4268.1642145,  9570.6489402, ..., 12013.888135 ,
       44955.6079538, 10844.4376388])

In [21]:
from sklearn.metrics import r2_score

In [22]:
lossy=r2_score(target_train,preds)

In [23]:
lossy

0.9757726664985225

In [24]:
predict=model.predict(input_test)

In [25]:
lossy2=r2_score(target_test,predict)

In [26]:
lossy2

0.8473063525029869

In [27]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 100,2)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['sqrt', 'log2'], 'max_depth': [1, 100], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']}


In [28]:
rf=RandomForestRegressor()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(inputs_train,target_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['squared_error',
                                                      'friedman_mse',
                                                      'absolute_error',
                                                      'poisson'],
                                        'max_depth': [1, 100],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [29]:
rf_randomcv.best_params_

{'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 100,
 'criterion': 'poisson'}

In [30]:
best_random_grid=rf_randomcv.best_estimator_

In [31]:
best_random_grid.fit(inputs_train,target_train)

RandomForestRegressor(criterion='poisson', max_depth=100, max_features='log2',
                      min_samples_split=5, n_estimators=1000)

In [32]:
pred=best_random_grid.predict(inputs_train)

In [33]:
pred

array([ 4282.37952352,  4876.03582617,  9839.92195102, ...,
       12221.45403881, 42482.50687484, 12180.04139718])

In [37]:
lossy=r2_score(pred,target_train)

In [38]:
lossy

0.9320554935186915

In [39]:
predict=best_random_grid.predict(input_test)

In [40]:
predict

array([11648.73305308,  5997.04382024, 27743.49867771, 11076.74711101,
       33129.90616604,  7066.52527963,  3204.80029267, 14637.7387171 ,
        5055.01107235, 11729.31860708, 19709.76956444,  8039.89395314,
        4580.80169157, 42028.11713876, 45618.05390743, 42174.74747137,
       10924.02163121, 41990.21046233,  9884.39400758, 23101.61547083,
        6287.21723262,  9985.56236631,  2612.44171141,  3295.12823327,
       12380.28030561, 12260.5727853 , 14113.82881395,  7111.34648815,
       11237.74530244,  3961.38599216,  8003.80074746, 12241.99000635,
        3418.82742216,  6456.74672944,  4359.64949929, 11248.08235508,
        3896.94209654,  8289.2062067 , 24832.25045733, 38751.03573132,
        6701.31278323,  4069.00463424, 13508.51030073, 13529.09392445,
        6948.63638607, 13485.84407643, 12187.30664773,  7255.98968231,
       39180.98434435,  6674.31794076, 14584.88314577,  3875.10296599,
        7740.87848184,  2184.65830997, 11839.62575932, 11783.85213451,
      

In [41]:
lossy2=r2_score(predict,target_test)

In [42]:
lossy2

0.8208409696457954

In [43]:
import numpy as np
def rmse(targets, predictions):
    return np.sqrt(np.mean(np.square(targets - predictions)))

In [45]:
train_rmse=rmse(target_train,preds)
test_rmse=rmse(target_test, predict)

In [46]:
train_rmse

1874.8742248252706

In [47]:
test_rmse

4593.238974209434